In [1]:
import itertools
import random
import ast
import pandas as pd
import numpy as np
from collections import defaultdict

In [2]:
metric_patterns = pd.read_csv('stim_conditions/combo_chart_6.csv')
simple_patterns = [[int(x) for x in ast.literal_eval(pattern)] for pattern in metric_patterns['simple'].dropna().tolist()]
complex_patterns = [[int(x) for x in ast.literal_eval(pattern)] for pattern in metric_patterns['complex'].dropna().tolist()]
simple_silence_patterns = [[int(x) for x in ast.literal_eval(pattern)] for pattern in metric_patterns['simple_silence'].dropna().tolist()]
complex_silence_patterns = [[int(x) for x in ast.literal_eval(pattern)] for pattern in metric_patterns['complex_silence'].dropna().tolist()]

In [3]:
# Function to generate all unique combinations of four words without repetitions
def generate_unique_combinations(words):
    return list(itertools.permutations(words, 4))

def arrange_syllables(unique_combinations, previous_combination, used_combinations, cycle_limit):
    random.shuffle(unique_combinations)  # Shuffle to ensure a random selection each time
    for combo in unique_combinations:
        # Check if we have used all combinations in this cycle
        if len(used_combinations) >= cycle_limit:
            used_combinations.clear()  # Reset for a new cycle

        # Ensure new combo is not the same as previous and does not start with previous combo's last element
        if combo != previous_combination \
           and (not previous_combination or combo[0] != previous_combination[-1]) \
           and combo not in used_combinations:
            return combo
    return None  # This should ideally not occur

def interweave(syllables, silence_pattern):
    # Ensure syllables are broken into individual letters
    flattened_syllables = [letter for word in syllables for letter in word]
    
    # Create the interweaved sequence
    interweaved = []
    for letter, silence in zip(flattened_syllables, silence_pattern):
        interweaved.append(letter)
        interweaved.append(silence)
    
    return interweaved
def pick_pattern_template_and_syllables(condition):

    # Choose appropriate patterns based on the condition
    patterns = simple_patterns if condition == 'simple' else complex_patterns
    silence_patterns = simple_silence_patterns if condition == 'simple' else complex_silence_patterns

    # Flatten the patterns if they are lists of lists
    patterns = [item for sublist in patterns for item in sublist]
    silence_patterns = [item for sublist in silence_patterns for item in sublist]

    # Randomly select 96 patterns and 96 silence patterns
    selected_patterns = random.choices(patterns, k=576)
    selected_silence_patterns = random.choices(silence_patterns, k=576)

    # Combine selected patterns into one string each
    pattern = ''.join([str(x) for x in selected_patterns])

    return pattern

In [4]:
words = ['ABC', 'DEF', 'GHI', 'JKL']
unique_combinations = generate_unique_combinations(words)
arranged_syllables = []
previous_combination = None
used_combinations = set()

cycle_limit = len(unique_combinations)

In [5]:
for _ in range(288):  # We aim for 48 sets
    new_set = arrange_syllables(unique_combinations, previous_combination, used_combinations, cycle_limit)
    if new_set:
        arranged_syllables.append(new_set)
        used_combinations.add(new_set)
        previous_combination = new_set
        # flatten the list of lists
flattened_syllables = [letter for word in arranged_syllables for letter in word]
print('flattened_syllables:', flattened_syllables)

flattened_syllables: ['DEF', 'GHI', 'JKL', 'ABC', 'DEF', 'GHI', 'ABC', 'JKL', 'DEF', 'ABC', 'GHI', 'JKL', 'ABC', 'DEF', 'GHI', 'JKL', 'ABC', 'GHI', 'DEF', 'JKL', 'GHI', 'DEF', 'ABC', 'JKL', 'DEF', 'ABC', 'JKL', 'GHI', 'ABC', 'GHI', 'JKL', 'DEF', 'ABC', 'DEF', 'JKL', 'GHI', 'ABC', 'JKL', 'GHI', 'DEF', 'JKL', 'ABC', 'GHI', 'DEF', 'GHI', 'DEF', 'JKL', 'ABC', 'GHI', 'JKL', 'DEF', 'ABC', 'GHI', 'ABC', 'DEF', 'JKL', 'GHI', 'JKL', 'ABC', 'DEF', 'JKL', 'DEF', 'GHI', 'ABC', 'GHI', 'ABC', 'JKL', 'DEF', 'ABC', 'JKL', 'DEF', 'GHI', 'JKL', 'ABC', 'DEF', 'GHI', 'DEF', 'JKL', 'ABC', 'GHI', 'DEF', 'JKL', 'GHI', 'ABC', 'JKL', 'GHI', 'DEF', 'ABC', 'JKL', 'DEF', 'ABC', 'GHI', 'JKL', 'GHI', 'ABC', 'DEF', 'JKL', 'GHI', 'DEF', 'ABC', 'DEF', 'ABC', 'GHI', 'JKL', 'DEF', 'JKL', 'ABC', 'GHI', 'ABC', 'JKL', 'GHI', 'DEF', 'ABC', 'GHI', 'JKL', 'DEF', 'ABC', 'JKL', 'DEF', 'GHI', 'ABC', 'GHI', 'DEF', 'JKL', 'DEF', 'JKL', 'GHI', 'ABC', 'DEF', 'GHI', 'ABC', 'JKL', 'DEF', 'ABC', 'JKL', 'GHI', 'JKL', 'DEF', 'GHI', 'ABC'

In [6]:

# Directly use the 'complex' sequence for analysis
interweaved_sequence = interweave(flattened_syllables, pick_pattern_template_and_syllables('simple'))

# Define the list of valid pairs from your specification
valid_pairs = [
    ('A', 'B'), ('B', 'C'), ('C', 'D'), ('C', 'G'), ('C', 'J'),
    ('D', 'E'), ('E', 'F'), ('F', 'G'), ('F', 'A'), ('F', 'J'),
    ('G', 'H'), ('H', 'I'), ('I', 'J'), ('I', 'A'), ('I', 'D'),
    ('J', 'K'), ('K', 'L'), ('L', 'A'), ('L', 'D'), ('L', 'G')
]

# Convert the list of valid pairs into a set for quick lookup
valid_pair_set = set(valid_pairs)

# Initialize the frequency table for transitional occurrences
pair_transition_freq = defaultdict(lambda: defaultdict(int))

# Regular counting for all but the last pair
for i in range(0, len(interweaved_sequence) - 2, 2):  # Step through individual pairs
    current_pair = (interweaved_sequence[i], interweaved_sequence[i + 2])
    # Only count if current pair is valid
    if current_pair in valid_pair_set:
        silence_value = interweaved_sequence[i + 1]
        pair_transition_freq[current_pair][silence_value] += 1

# Special handling for the last element transitioning back to the first:
# This checks if such a pair (last-first) is a valid pair to be counted
if len(interweaved_sequence) > 2:  # Ensure there are enough elements for a wrap-around
    last_pair = (interweaved_sequence[-2], interweaved_sequence[0])
    last_silence = interweaved_sequence[-1]
    if last_pair in valid_pair_set:
        pair_transition_freq[last_pair][last_silence] += 1

# Convert the frequency table to a DataFrame for analysis and display
df_pair_transitions = pd.DataFrame(pair_transition_freq).T.fillna(0).astype(int)
df_pair_transitions['Total'] = df_pair_transitions.sum(axis=1)
df_pair_transitions.sort_values('Total', ascending=False, inplace=True)
df_pair_transitions = df_pair_transitions.sort_index(axis=1)  # Sorting for better visualization and consistency

# Display the refined frequency table
print("Pair Transitional Frequencies")
display(df_pair_transitions)


Pair Transitional Frequencies


,,1,2,3,4,Total
D,E,20,18,7,3,48
J,K,24,11,5,8,48
E,F,14,15,15,4,48
A,B,19,15,7,7,48
K,L,22,12,9,5,48
B,C,21,7,12,8,48
H,I,21,12,8,7,48
G,H,19,13,9,7,48
L,D,9,4,3,2,18
C,G,6,5,5,2,18


In [7]:
# Initialize the frequency table for letter pairs with their transitional silence value
pair_transition_freq = defaultdict(lambda: defaultdict(int))

# Directly use the 'complex' sequence for analysis
interweaved_sequence = interweave(flattened_syllables, pick_pattern_template_and_syllables('complex'))  # If 'complex' is the interweaved sequence

# Regular counting for all but the last pair
for i in range(0, len(interweaved_sequence) - 2, 2):  # Step through individual pairs
    current_pair = (interweaved_sequence[i], interweaved_sequence[i + 2])
    # Only count if current pair is valid
    if current_pair in valid_pair_set:
        silence_value = interweaved_sequence[i + 1]
        pair_transition_freq[current_pair][silence_value] += 1

# Special handling for the last element transitioning back to the first:
# This checks if such a pair (last-first) is a valid pair to be counted
if len(interweaved_sequence) > 2:  # Ensure there are enough elements for a wrap-around
    last_pair = (interweaved_sequence[-2], interweaved_sequence[0])
    last_silence = interweaved_sequence[-1]
    if last_pair in valid_pair_set:
        pair_transition_freq[last_pair][last_silence] += 1

# Convert the frequency table to a DataFrame for analysis and display
df_pair_transitions = pd.DataFrame(pair_transition_freq).T.fillna(0).astype(int)
df_pair_transitions['Total'] = df_pair_transitions.sum(axis=1)
df_pair_transitions.sort_values('Total', ascending=False, inplace=True)
df_pair_transitions = df_pair_transitions.sort_index(axis=1)  # Sorting for better visualization and consistency

# Display the refined frequency table
print("Pair Transitional Frequencies")
display(df_pair_transitions)

Pair Transitional Frequencies


,,1,2,3,4,Total
D,E,18,13,10,7,48
J,K,19,15,11,3,48
E,F,13,14,14,7,48
A,B,24,13,4,7,48
K,L,20,19,5,4,48
B,C,15,18,8,7,48
H,I,20,16,8,4,48
G,H,25,14,7,2,48
L,D,8,5,3,2,18
C,G,8,2,3,5,18
